# Year 1 Semester 2 Research Diary

------------------------------------------------------

## Until 02/08/2017

The produced code is able to create white noise and convolute it with CMB data. The resulting map is close to what is observed. Efforts to recreate a scan strategy and dissect the map into bolometer signals have been made. However when recompressed into a map there is a substantial difference to the original map, indicating an error somewhere during the dissection and reconstrusction.

The code to create the CMB map works in the following way:

1. Define map parameters. We choose a square map of 2048" sidelength.
1. Read in CMB data. Specifically target $l$ and $D_l$ values, which are the first and second column of the provided in the file *for_lennart/plik_plus_r0p01_highell_lensedtotCls.dat* respectively. Then use $C_l = \frac{2 \pi D_l}{l(l+1)}$ to get $C_l$ values for further analysis.
1. Turn obtained $C_l$ values into a 2D array.
    1. Create wavevectors along each dimension for provided map parameters.
    1. Tile wavevectors to get 2D versions.
    1. Use $C_l = \sqrt{k_x^2+k_y^2}$ to populate two dimensional $C_l$ array.
1. Create noise in Fourier space.
1. Add signal via $\mathrm{CMB} = \sqrt{C_l \frac{df}{2}} \times (\mathrm{realnoise} + i \times \mathrm{imaginarynoise})$
1. Inverse FFT to get the real map, but only keep a quarter

The code produces CMB maps that look like the following. Note that creating the 2D version of $C_l$ is computationally expensive and hence the array was only computed once, then saved and is now loaded. Since this is purely the CMB signal and does not contain noise yet this is adequate.

<img src = "../cosmologydigitisation/Plots/21082017/cmbnoisefourier.png">

## 03/08/2017

Meeting with Christian to discuss progress. Guidance on when to implement noise provided: either at the bolometer stage or when creating the map from its Fourier Transform, but not both.

For the next meeting aim to:
* Be able to recreate maps accurately, after decomposing them into bolometer signals and recombining them.
* Check Parseval's Theorem.

Agreed to schedule meetings more regularly and set clearer expectations.

## 08/08/2017 - 09/08/2017

Data intensive astronomy workshop ADACS.

## 10/08/2017

Completed Version Control setup on personal laptop and university computer.

# 12/08/2017

## Parseval's theorem

We would like to confirm Parseval's theorem, which states that power is conserved when moving between real and frequency space. Expressed mathematically:

$$\sum_n |x(n)|^2 = \frac{1}{N} \sum_n |X(n)|^2,$$

where $X(n)$ is the discrete Fourier transform of $x(n)$, both of length $N$. In our case we must carry out the summation over the entire image, i.e. two dimensions.

Straightforward implementation of this confirms this for the CMB map. We immediately check this after applying the inverse Fourier transform. Confirmation of the above is less trivial for the finished map, as we cut the inverse Fourier transform output into a quarter. To check that Parseval's theorem still holds we draw many noise realisations, apply the inverse Fourier transform and compare the power to their corresponding maps in frequency space.



```
psvindex = range(100)
powdiff = []

for i in psvindex:
    realpart = factor * normal(size=(pixelnumber, pixelnumber))
    imagpart = factor * normal(size=(pixelnumber, pixelnumber))
    cmbnoisefreqspace = (realpart + 1j * imagpart)
    cmbnoisemap = fft.ifft2(cmbnoisefreqspace)[0:int(pixelnumber/2), 0:int(pixelnumber/2)]
    psvfreq = real(sum(cmbnoisefreqspace * conjugate(cmbnoisefreqspace)))
    psvreal = real(sum(cmbnoisemap * conjugate(cmbnoisemap))) * 4 * pixelnumber**2
    powdiff.append((psvfreq-psvreal)/psvfreq)
```

We define $\Delta P = (P_f-P_r)/P_f $ where $P$ indicates power and $f$ and $r$ Fourier and real space respectively. We plot $\Delta P$ and show mean and standard deviation thereof below.

<img src = "../cosmologydigitisation/Plots/12082017/relativepowerdifference.png">

From the plot we see that the powers are consistent over many iterations, albeit the crude cutting of the real space map. We note that we must perform this check before throwing away the imaginary component of the real space map.

We now check if power is conserved in Srini's powerspectrum code. This is not the case. We spot that the Fourier transform of the image is multiplied by the radial pixel resolution, which changes the power. We remove this extra factor and observe that Parseval's theorem is restored.

To check whether power is conserved during the radial averaging process we must retain information about the number of pixels in each radial bin. We then check Parseval's theorem by multiplying each radial power by its bincount and then summing the results. We achieve a result that is appreciably close to the original power, but nevertheless different. Redefining $\Delta P = (P_0-P_r)/P_0$ and drawing 100 realisations we observe the behaviour displayed below.

<img src = "../cosmologydigitisation/Plots/12082017/powerdiffradial.png">

The only explanation we can think of is that this is an effect due to lying circular bins over a rectangular image.

## Recreating Scan Strategies appropriately

We have previously found inconsistencies between a map and a copy of it, which was broken down into bolometer signals and then recompressed. We investigate this difference now.

Recreation and Compression code runs extremely slow on laptop. Unsure how to improve this. We need to iterate over every pixel in the map to break it up into bolometer signals. We need to iterate over all bolometer signals to compress them correctly into the map again. We are unsure if taking the resolution down is appropriate, since it may not capture all CMB features appropriately. To check if the code is correct we take the number of bolometer signals that correspond to a map pixel down from 4000 to 120. This is done by changing the readout frequency from $500\mathrm{Hz}$ to $6\mathrm{Hz}$. We then formulate $\Delta M = (M_0-M_r)/M_0$ where $M_0$ are elements of the original map and $M_r$ are elements of the map that has been expanded and compressed. The result is distinctly non-zero. However, $\Delta M$ is very small and it appears to look like white noise. We check this by calculating its powerspectrum, which confirms this intuition. The relevant plots are shown below. Furthermore we confirm that the average of $\Delta M$ is consistent with zero within one standard deviation.

We therefore have confidence that the code written is correct. Perhaps the error experienced was due to including additional noise when recreating the scan strategy. Without doing this we are however unsure how a faithful digitisation would work that operates at timestream level.

<img src = "../cosmologydigitisation/Plots/12082017/mapdiff.png">

<img src = "../cosmologydigitisation/Plots/12082017/mapdiffpower.png">

# 15/08/2017

Meeting with Christian. Discussed progress on Parseval's theorem and recreating the scan strategy. On Scan strategy everything's good, we're limited by `float64` precision. On Parseval's theorem a little puzzled by the difference induced when doing the radial averaging. Some issues are:

* Power leaking into higher modes
* Not all data points are hit by circular apertures

For the next meeting adjust:

* Mean of the Map = 0 $\checkmark$
* Check if apertures cover all of the map
$\checkmark$ <font color = "Grey"> They did not. This was due to the maximum bin not being set correctly. It used the maximum of the 1D wavevector, rather than the maximum of $\sqrt{l_x^2+l_y^2}$. Adjusted this in a hacky way in Srini's code. After adjusting this Parseval's theorem is restored in the radial averaging.</font>
* Apply a mask to the fft map and zero pad, account for this in PS by multiplying it by $1/\mathrm{avg}(\mathrm{mask}^2)$
$\checkmark$ <font color = "Grey"> Wrote a zeropad and a cosine mask function in python. cos mask is written as array multiplication to save time.</font>
* Change noise to be induced at the time stream level, not in the first fourier transform instance (Noise is added by addition)
    * Began work on this, creating the noise and adding it is trivial, but the raw cmb map does not look right. Powerspectrum doesnt look like it should either. But this is not fault of the powerspectrum code. The PS for the raw CMB image and one recreated from the tod without adding noise have the same PS. Therefore we must look at the creation of the raw cmb image.
    * whats going on with the top left corner in the pure cmb map? Should this be the centre, but fftshift? on the $k_{x, y}$ values when creating CL2D maybe?
    * Does not seem mathematically trivial that the two stages of noise addition should be equivalent
* Increase map size to 20x20 degrees $\checkmark$ <font color = "Grey">Actually have a 34x34 degree field at first, with 2 arcmin resolution.</font>
* Compare powerspectrum in and powerspectrum out. Recreation: $\checkmark$ <font color = "Grey">Leave rest of analysis until noise is sorted out.</font>

$\rightarrow$ Implementing the noise is challenging. While it seems conceptually logical that it should not matter whether we add noise in frequency or real space, they are not mathematically interchangable. The matching operation in real space is convolution. Perhaps for white noise however the convolution and addition are equivalent? This does not seem right however. This should be discussed during the next meeting. Until then, the relevant plots are below.

IFFT of the pure CMB data, i.e. without noise.
<img src = "../cosmologydigitisation/Plots/21082017/cmbrealspace.png">
Noise via addition.
<img src = "../cosmologydigitisation/Plots/21082017/cmbnoiseaddedinreal.png">
Noise via concolution.
<img src = "../cosmologydigitisation/Plots/21082017/cmbnoisebyconvolution.png">

We notice that there are some artifacts to the convolution (stringiness), but it matches the desired features for a mock observation.

Attempted at recreating the input Powerspectrum. Unsure whether to use $P_k$ or $P_k k(k+1)$. Neither shapes look correct.

<img src = "../cosmologydigitisation/Plots/21082017/pspk.png">
<img src = "../cosmologydigitisation/Plots/21082017/pscl.png">

## 25/08/2017

Meeting with Christian. The mock observations is the addition of two gaussian random fields. The CMB and a Noise field. So the CMBNoise field must be added to another random field. This addition can either take place in real space or Fourier space. For good comparison we want to add 1 observed CMB map to N Noise maps to produce N observations. Then coadd (average, dont worry about weighting for now) the N produced maps and calculate the powerspectrum. When adding the noise we can potentially digitise.

* How to set average & standard deviation of added noise? For now set average to 0 and standard deviation to the standard deviation of the CMB map.
* Wrote code to create a CMB map, break it up into tod, create N mockobservations with added noise (can digitise here), then recompress into map, compare powerspectrum
* Noticed white noise is not being displayed appropriately by powerspectrum code, this is due to applying the cosine mask

The CMB maps PS is compared to the average of 10 maps with added noise below.

<img src = "../cosmologydigitisation/Plots/31082017/pscompare.png">


## 31/08/2017

Meeting with Christian. Needed to Apply cosine mask and zeropadding in real space, not Fourier space. This fixed the strange appearance of the white noise. For the next meeting aim to:

* Fix scale of noise. Want $6\mu \mathrm{K}$ per pixel. $\checkmark$ Implemented this such that this corresponds to the variance of the noise added to the cmb at timestream level. Does this change if we move to N obervations?
* Substract CMB map and the averaged noise map to see the difference in real space $\checkmark$ This is done for 50 obervations and the result shown below

<img src = "../cosmologydigitisation/Plots/05092017/cmbnoisecomparemap.png">

* Plot Powerspectra $\checkmark$ this is again done for 50 noise maps

<img src = "../cosmologydigitisation/Plots/05092017/cmbnoisecompareps.png">

The difference has a mean of 0 within one $\sigma$.

* Digitise and compare powerspectra
* Use profiler to analyse where the code needs to be improved (see cprofiler, pstats for this)
* (See what happens when we add non-white noise)